In [58]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import imdb
from googlesearch import search
import re
import nltk
from nltk.corpus import stopwords
import string
import collections

In [2]:
df = pd.read_csv('Netflix_data.csv')
df.head()

,Title,Bag_of_words
0,10 Cloverfield Lane,theres convinces woman apocalyptic accident bu...
1,10 Days in Sun City,unexpected south nigeria faces joins pageant w...
2,10 jours en or,boy trip lives left embark young carefree road...
3,100 Days Of Solitude (100 días de soledad),alone connecting photographer spanish beauty s...
4,100 Metros,sclerosis multiple fatherinlaws cranky triathl...


In [174]:
def movie_recommendation(movie_name):
    
    counter = CountVectorizer(encoding='utf-8-sig')
    movie_name = movie_name + " "
    
    if sum(df['Title'] == movie_name) >= 1:
        
        matrix = counter.fit_transform(df['Bag_of_words'])
        similarity = cosine_similarity(matrix)
        
        recommendations = []
        
        index = df[df['Title'] == movie_name].index.tolist()[-1] 
        sort_index = np.argsort(-1*similarity[index])

        for num in sort_index[:10]:

            recommendations.append(df['Title'][num])
            
    else:
        
        web_url = []
        
        for j in search(movie_name, tld="com", num=30, stop=30, pause=2):
            web_url.append(j)
            
        for num in range(len(web_url)):
            string = re.findall(r'https://www.imdb.com/'+'(.*)',web_url[num])

            if string == []:
                continue
            else:
                break
    
        if string == []:
            imdb_id = 'No ID found'
        else:
            imdb_id = re.findall(r'[0-9]+',string.pop()).pop()
            
        imdb_movie = imdb.IMDb()
        directors_list = []
        actors_list = []
        genres_list = []
        languages_list = []
        
        if imdb_id == 'No ID found':
            recommendations = 'Please enter another title'
            
        else:
            movie_obj = imdb_movie.get_movie(imdb_id)
            
            try:
                directors = []
                for director in movie_obj['directors']:
                    directors.append(str(director))
                directors = ", ".join(directors)
            
            except:
                directors = 'NAN'

            try:
                actors = []
                for actor in movie_obj['cast']:
                    actors.append(str(actor))
                actors = ", ".join(actors)

            except:
                actors = 'NAN'

            try:
                genres = []
                for genre in movie_obj['genres']:
                    genres.append(str(genre))
                genres = ", ".join(genres)

            except:
                genres = 'NAN'

            try:
                languages = []
                for language in movie_obj['languages']:
                    languages.append(str(language))
                languages = ", ".join(languages)

            except:
                languages = 'NAN'
                
            try:
                plot = movie_obj.get('plot')[0]
            except:
                plot = 'NAN'


            directors_list.append(directors)
            actors_list.append(actors)
            genres_list.append(genres)
            languages_list.append(languages)
            
            import string
            plot_list = plot.translate(str.maketrans('', '', string.punctuation)).split()
            
            list_plot = [word.lower() for word in plot_list if word.lower() not in stopwords.words('english')]
            list_plot = set(list_plot)
            plot_line = ", ".join(list_plot)
            
            edited_dir = [word.replace(" ","").lower() for word in directors.split(',')]
            edited_dir = ", ".join(edited_dir)
            
            edited_act = [word.replace(" ","").lower() for word in actors.split(',')]
            edited_act = ", ".join(edited_act)
            
            edited_genre = genres.lower()
            edited_language = languages.lower()
            
            bag_of_word = plot_line + ", " + edited_dir + ", " + edited_act + ", " + edited_genre
            bag_of_words = bag_of_word.replace(", "," ")
            
            bow = list(df['Bag_of_words'])
            bow.append(bag_of_words)
            
            matrix = counter.fit_transform(bow)
            similarity = cosine_similarity(matrix)
        
            recommendations = []
 
            sort_index = np.argsort(-1*similarity[-1])

            for num in sort_index[1:11]:

                recommendations.append(df['Title'][num])
     
    return recommendations

In [190]:
movie_recommendation('Avengers infinity war')

['End Game ',
 'Captain America: Civil War ',
 'Avengers: Age of Ultron ',
 'LEGO Marvel Super Heroes: Guardians of the Galaxy ',
 'Spider-Man: Homecoming ',
 'Thor ',
 'Power Rangers Super Samurai: Trickster Treat ',
 'Satria Heroes: Revenge of the Darkness ',
 'Marvel Super Hero Adventures: Frost Fight! ',
 'Jurassic Park ']

In [192]:
movie_recommendation('Get out')

['May the Devil Take You ',
 'I Am the Pretty Thing That Lives in the House ',
 'Krishna Cottage ',
 'The Shallows ',
 'Unfriended: Dark Web ',
 'Eli ',
 'Unruly Friends (Bartita) ',
 'Stephanie ',
 'The Body ',
 "Gerald's Game "]